#SV3D: Novel Multi-view Synthesis and 3D Generation from a Single Image using Latent Video Diffusion

The model can be found on [GitHub](https://github.com/Stability-AI/generative-models). The checkpoint is located on their [HugginFace](https://huggingface.co/stabilityai/sv3d/tree/main).

The script *simple_video_sample.py* has been adapted to be used in this Jupyter notebook

##Installation and setup

In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118

!pip install transformers einops opencv-python matplotli
!pip install transformers einops rembg fire omegaconf safetensors imageio
!pip install opencv-python-headless  # to avoid GUI-related errors in Colab
!pip install onnxruntime-gpu
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch_lightning
!pip install kornia
!pip install open-clip-torch

!pip uninstall -y imwatermark imhist
!pip install git+https://github.com/guofei9987/imwatermark.git


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 574.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-pmhvkl10
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-pmhvkl10
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=0106559eded263e28210a79f63004b05c69b402cdd4a0fbe8988fee3f29b5f49
  Stored in directory: /tmp/pip-ephem-wheel-cache-erzggf01/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB

In [ ]:
# Clone the StabilityAI repo
!git clone https://github.com/Stability-AI/generative-models
%cd generative-models

Cloning into 'generative-models'...
remote: Enumerating objects: 1119, done.
remote: Counting objects: 100% (554/554), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1119 (delta 428), reused 412 (delta 384), pack-reused 565 (from 1)
Receiving objects: 100% (1119/1119), 86.66 MiB | 15.54 MiB/s, done.
Resolving deltas: 100% (589/589), done.
/content/generative-models


##Load the check point

In [ ]:
import requests

# Create checkpoint directory
import os
os.makedirs("checkpoints", exist_ok=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
checkpoint_path = "/content/drive/MyDrive/0-Master/TAVA_NVS/sv3d_u.safetensors"

import os

if os.path.exists(checkpoint_path):
    print("Found checkpoint!")
else:
    print("Checkpoint not found. Double-check the path.")


Found checkpoint!


In [ ]:
target_path = "/content/generative-models/checkpoints/sv3d_u.safetensors"

# Create checkpoints folder if it doesn't exist
os.makedirs(os.path.dirname(target_path), exist_ok=True)

# Copy from Drive to repo
import shutil
shutil.copy(checkpoint_path, target_path)

print("Copied to:", target_path)


Copied to: /content/generative-models/checkpoints/sv3d_u.safetensors


##Load Input

##Run the model

In [ ]:
!sed -i '/from imwatermark import WatermarkEncoder/d' sgm/inference/helpers.py
!sed -i 's/samples = embed_watermark(samples)/# samples = embed_watermark(samples)/' scripts/sampling/simple_video_sample.py
!sed -i '/self.encoder = WatermarkEncoder()/d' sgm/inference/helpers.py
!sed -i '/self.encoder.set_watermark("bits", self.watermark)/d' sgm/inference/helpers.py
!sed -i 's/    def __init__(self, watermark):/    def __init__(self, watermark):\n        pass/' sgm/inference/helpers.py
!sed -i 's/    def __call__(self, images):/    def __call__(self, images):\n        return images/' sgm/inference/helpers.py


In [ ]:
import sys
sys.path.append('.')  # ensure base dir is in path

# Create a wrapper to run from notebook
from scripts.sampling.simple_video_sample import sample




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
input_path = "/content/drive/MyDrive/0-Master/TAVA_NVS/0.jpg"

sample(
    input_path=input_path,
    version="sv3d_u",
    device="cpu",
    decoding_t=1,              # 1 frame at a time (least memory)
    motion_bucket_id=64,      # lower motion = less compute
    fps_id=6,                  # moderate frame rate
)

VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


...

VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Initialized embedder #0: FrozenOpenCLIPImagePredictionEmbedder with 683800065 params. Trainable: False
Initialized embedder #1: VideoPredictionEmbedderWithEncoder with 83653863 params. Trainable: False
Initialized embedder #2: ConcatTimestepEmbedderND with 0 params. Trainable: False
Restored from checkpoints/sv3d_u.safetensors with 0 missing and 0 unexpected keys


100%|███████████████████████████████████████| 890M/890M [00:58<00:00, 16.0MiB/s]
